In [12]:
import glob
import cv2
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
from tqdm import tqdm

from deepcell_spots.simulate_data import gaussian_spot_image_generator
from deepcell_spots.preprocessing_utils import min_max_normalize

In [2]:
import random
random.seed(0)

In [3]:
data_folder = '../data/simulated_images/'
image_files = glob.glob(data_folder + 'background*')
im = np.array(Image.open(image_files[0]))

plt.imshow(im)
plt.show()

['/data/users/emily/SeqFISH/for_simulated_spots/MAX_20200702-SeqFISH-HEK293T17_0nMp_50nMs.nd2 - C=2.tif',
 '/data/users/emily/SeqFISH/for_simulated_spots/MAX_20200702-SeqFISH-HEK293T17_0nMp_50nMs001.nd2 - C=2.tif',
 '/data/users/emily/SeqFISH/for_simulated_spots/MAX_20200702-SeqFISH-HEK293T17_0nMp_50nMs.nd2 - C=1.tif']

In [7]:
im = im[:1280, :1280]

im_stack = []
dim_output = 128
# Get dimensions of existing stack
dim_x = np.shape(im)[1]
dim_y = np.shape(im)[0]

# Calculate the number of slices of that size in loaded image
x_slices = int(dim_x/dim_output)
y_slices = int(dim_y/dim_output)

# Create sliced images from loaded image and add to stack
for i in range(x_slices):
    for ii in range(y_slices):
        im_slice = im[ii*dim_output:(ii+1)*dim_output,i*dim_output:(i+1)*dim_output]
        im_stack.append(im_slice)

im_stack = np.expand_dims(im_stack, axis=-1)
np.shape(im_stack)

(100, 128, 128, 1)

In [8]:
L = 128 # square image side length
sigma_mean = 1.5 # mean sigma of gaussian dot
sigma_std = 0.1 # deviation from sigma
A_std = 0 # deviation of intensity of gaussian
noise_mean = 0 # white noise added to entire image - mean and std
noise_std = 0

In [13]:
sim_images_density = {}
sim_coords_density = {}
num_spots_list = np.linspace(20,300,6).astype(int)
A_list = np.linspace(3,6,6)
num_images = np.shape(im_stack)[0]
for i in tqdm(range(len(num_spots_list))):
    g = gaussian_spot_image_generator(L, num_spots_list[i], num_spots_list[i], 
                                      sigma_mean, sigma_std, 
                                      A_list[2], A_std, 
                                      noise_mean = noise_mean, noise_std = noise_std,
                                      segmask=True,yield_pos=True)
    X = []
    y = []
    for ind in range(num_images):
        img, label, x_list, y_list, bboxes = next(g)
        dot_coords = np.array([y_list, x_list]).T

        X.append(np.expand_dims(img,axis=-1))
        y.append(dot_coords)

    X = np.array(X)
    
    sim_spot_stack = np.array([cv2.GaussianBlur(im_stack[i,...,0],(7,7),0) + X[i,...,0]*20 for i in range(num_images)])
    sim_spot_stack = np.expand_dims(sim_spot_stack, axis=-1)
    sim_spot_stack = min_max_normalize(sim_spot_stack)
    
    sim_images_density[num_spots_list[i]] = sim_spot_stack
    sim_coords_density[num_spots_list[i]] = y

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 6/6 [01:44<00:00, 17.38s/it]


In [14]:
sim_images_intensity = {}
sim_coords_intensity = {}
num_spots_list = np.linspace(20,300,6).astype(int)
A_list = np.linspace(3,6,6)
num_images = np.shape(im_stack)[0]
for i in tqdm(range(len(A_list))):
    g = gaussian_spot_image_generator(L, num_spots_list[2], num_spots_list[2], 
                                      sigma_mean, sigma_std, 
                                      A_list[i], A_std, 
                                      noise_mean = noise_mean, noise_std = noise_std,
                                      segmask=True,yield_pos=True)
    X = []
    y = []
    for ind in range(num_images):
        img, label, x_list, y_list, bboxes = next(g)
        dot_coords = np.array([y_list, x_list]).T

        X.append(np.expand_dims(img,axis=-1))
        y.append(dot_coords)

    X = np.array(X)
    
    sim_spot_stack = np.array([cv2.GaussianBlur(im_stack[i,...,0],(7,7),0) + X[i,...,0]*20 for i in range(num_images)])
    sim_spot_stack = np.expand_dims(sim_spot_stack, axis=-1)
    sim_spot_stack = min_max_normalize(sim_spot_stack)
    
    sim_images_intensity[A_list[i]] = sim_spot_stack
    sim_coords_intensity[A_list[i]] = y

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 6/6 [01:25<00:00, 14.22s/it]


In [15]:
np.savez('../data/sim_images_density.npz',
         X=list(sim_images_density.values()),
         y=list(sim_coords_density.values()))

np.savez('../data/simulated_images/sim_images_intensity.npz',
         X=list(sim_images_intensity.values()),
         y=list(sim_coords_intensity.values()))

/usr/local/lib/python3.8/dist-packages/numpy/lib/npyio.py:713: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  val = np.asanyarray(val)
